In [28]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import json
import torch.utils.data as data

In [ ]:
#With heavy help from https://github.com/benmoseley/harmonic-oscillator-pinn-workshop/blob/main/PINN_intro_workshop.ipynb

In [3]:
class FeedForward(nn.Module):
    #So bog basic it should tap for black mana
    #DT That one's for you
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(8, 30)
        self.linear2 = nn.Linear(30,30)
        self.linear3 = nn.Linear(30,30)
        self.linear4 = nn.Linear(30,8)
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        x = self.linear4(x)
        return x

In [ ]:
g = 9.81
def derivatives(t, state):
    L1, L2, m1, m2, theta1, z1, theta2, z2 = state
    delta = theta2 - theta1

    denominator1 = (m1 + m2) * L1 - m2 * L1 * np.cos(delta) ** 2
    denominator2 = (L2 / L1) * denominator1

    dtheta1_dt = z1
    dz1_dt = (
        (m2 * L1 * z1 ** 2 * np.sin(delta) * np.cos(delta)
         + m2 * g * np.sin(theta2) * np.cos(delta)
         + m2 * L2 * z2 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta1))
        / denominator1
    )
    dtheta2_dt = z2
    dz2_dt = (
        (-m2 * L2 * z2 ** 2 * np.sin(delta) * np.cos(delta)
         + (m1 + m2) * g * np.sin(theta1) * np.cos(delta)
         - (m1 + m2) * L1 * z1 ** 2 * np.sin(delta)
         - (m1 + m2) * g * np.sin(theta2))
        / denominator2
    )

    return np.array([0,0,0,0,dtheta1_dt, dz1_dt, dtheta2_dt, dz2_dt])

In [11]:
#Get the data into a nice form
with open('../data_folder/raw_data.json', 'r') as file:
    raw_data = json.load(file)

In [47]:
#Gonna want inputs, a list of 8d vectors; and outputs, a list of 8d vectors that come immediately after inputs
inputs = []
outputs = []
np.random.seed(522)
ts_indices = np.random.choice(len(raw_data), replace=False, size=len(raw_data)//4)
#Use a quarter of all time series generated 'cause there's a lot
for i in ts_indices:
    timeseries = raw_data[i]
    n_tsteps = timeseries['num_tsteps']
    #Also, use a quarter of all time steps
    tstep_indices = np.random.choice(n_tsteps-1, replace=False, size=n_tsteps//4)
    input_vecs = np.empty((len(tstep_indices), 8))
    output_vecs = np.empty((len(tstep_indices), 8))
    for j, k in enumerate(tstep_indices):
        input_vecs[j, 4:] = np.array([timeseries['theta1'][k], timeseries['theta2'][k], timeseries['theta1prime'][k], timeseries['theta2prime'][k]])
        output_vecs[j, 4:] = np.array([timeseries['theta1'][k+1], timeseries['theta2'][k+1], timeseries['theta1prime'][k+1], timeseries['theta2prime'][k+1]])
    input_vecs[:, :4] = np.array([timeseries['length1'], timeseries['length2'], timeseries['mass1'], timeseries['mass2']])
    output_vecs[:, :4] = np.array([timeseries['length1'], timeseries['length2'], timeseries['mass1'], timeseries['mass2']])
    for j in range(len(input_vecs)):
        inputs.append(list(input_vecs[j]))
        outputs.append(list(output_vecs[j]))
    
test_indices = np.random.choice(len(inputs), replace=False, size = len(inputs)//5)
remaining_indices = set(range(len(inputs))) - set(test_indices)
val_indices = np.random.choice(list(remaining_indices), replace=False, size=len(inputs)//5)
train_indices = list(set(range(len(inputs))) - set(test_indices) - set(val_indices))

assert set(train_indices).union(set(test_indices)).union(set(val_indices)) == set(range(len(inputs)))
assert len(set(train_indices).intersection(set(test_indices))) == 0
assert len(set(train_indices).intersection(set(val_indices))) == 0
assert len(set(test_indices).intersection(set(val_indices))) == 0

In [44]:
train_X = torch.tensor(np.array(inputs)[train_indices])
train_y = torch.tensor(np.array(outputs)[train_indices])
test_X = torch.tensor(np.array(inputs)[test_indices])
test_y = torch.tensor(np.array(outputs)[test_indices])
val_X = torch.tensor(np.array(inputs)[val_indices])
val_y = torch.tensor(np.array(outputs)[val_indices])

batch_size = 8

train_dataset = data.TensorDataset(train_X, train_y)
test_dataset = data.TensorDataset(test_X, test_y)
val_dataset = data.TensorDataset(val_X, val_y)
train_loader = data.DataLoader(train_dataset, shuffle=False, batch_size=batch_size) #Don't need to shuffle 'cause it's already randomly ordered
test_loader = data.DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
val_loader = data.DataLoader(val_dataset, shuffle=False, batch_size=batch_size)
#Make validation data too?

In [ ]:
timestep = 1/200
pinn = FeedForward()
optimizer = torch.optim.Adam(pinn.parameters(), lr=1e-4)
num_epochs=10000
lamb=1/2 #Test different values for this?
for i in range(num_epochs):
    pinn.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output_pred = pinn(X_batch)
        derivs = (output_pred - X_batch) / 
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    #Gotta figure out the data-loading at some point...
    #pinn() is supposed to approximate, as a function, the next timestep given the state of a system, and training data with consistent timesteps

    #Gotta iterate over data loader somewhere in here
    #derivs = (output_vector - input_data) / timestep
    # physics_loss = (derivs - derivatives(input_data))**2 #Divide by N elns?
    #data_loss = lamb*(true_output - output_vector)**2
    #loss = physics_loss + data_loss
    #loss.backward() etc
    #optimizer.step()
    #Get some validation or something to plot idk